# Integration!
This script combines all parts of the Truth Model, State Estimation, and POMDP

In [2]:
import numpy as np
from HCW import c2d_van_loan, hcw_continuous_AB, Qd_from_accel_white

from estimation import KalmanFilter, State
from pomdp import POMDP, MCTS
from HCW import ThrusterParams, SatParams, NoiseParams, SimParams, HCWDynamics_3DOF, simulate_step, SimType

In [3]:
# Initializing simulation truth model

dt = 0.1
n_steps = 10000

# Satellite
main_thruster = ThrusterParams(5.0, 1)
rcs_thruster = ThrusterParams(0.2, 0.001)
sat = SatParams(mass_kg=500, thrusters=[main_thruster, rcs_thruster])

# Simulation
noise = NoiseParams(sigma_accel=0.1, sigma_pos=10, sigma_vel=2)
sim = SimParams(dt=dt, n_steps=n_steps, noise=noise, mean_motion_rad_s=0.0011)

x0 = np.array([500.0, 0.0, 0.0,
                0.0,   0.0, 0.0])

# Measurement Model 

# Position-only measurements --> state [x,y,z,vx,vy,vz] to measurements [x,y,z]
# y_k = [x, y, z]^T + noise
# H picks out just the position part from [x, y, z, xdot, ydot, zdot]^T
H = np.hstack([np.eye(3), np.zeros((3, 3))])

# Measurement noise covariance for position-only sensor
# Each axis has variance sigma_p^2, no cross-correlation.
R = (noise.sigma_pos**2) * np.eye(3)

hcw_dynamics = HCWDynamics_3DOF(sim, sat, x0)
X, Y, Y_meas, err, U_meas = hcw_dynamics.simulate(H, R, rng_seed=12, simtype=SimType.mass_varying_with_thrust)

Created simulation lasting 1000.00 sec (16.67 min) with 10000 steps
Simulated!


In [4]:
# Initialize Kalman Filter

x0 = np.array([500.0, 0.0, 0.0, 0.0, 0.0, 0.0])
P0 = np.diag(np.array([10, 10, 10, 5, 5, 5]))

# Model
n  = 0.0011
Ad, Bd = c2d_van_loan(*hcw_continuous_AB(sim.n), sim.dt)
Qd = Qd_from_accel_white(sim.dt, sigma_a = 0.01)

# Noise
R = .1 * np.eye(3) * 100 # Since 2 measurements

linear_ekf = KalmanFilter(Ad, Bd, H, Qd, R)
state = State(x0, P0)

In [ ]:
# Initialize POMDP
thrust = 0.1 # N (kg m/s2)
actions_6d = [
    (thrust, 0, 0), (-thrust, 0, 0),
    (0, thrust, 0), (0, -thrust, 0),
    (0, 0, thrust), (0, 0, -thrust),
    (0, 0, 0),  # coast
]

problem_6d = POMDP(gamma=0.95, states=None, actions=actions_6d, observations=None,
                   transition=None, reward=None, observation=None, tro=tro_6d)
